In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time, os,requests
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

video_url = 'https://www.instagram.com/emmafituk_/?hl=en'
#create results folder if does not exists
res_folder=os.getcwd()+'/results'
os.makedirs(res_folder, exist_ok=True)
#initiate webdriver
chrome_options = Options()
#chrome_options.add_argument("--headless")  # Optional: run in headless mode
prefs = {"download.default_directory" : res_folder}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument("--headless=new")  # Use the newer headless mode
chrome_options.add_argument("--window-size=1920,1080")  # Set a large enough window size
chrome_options.add_argument("--disable-gpu")  # Sometimes useful in headless
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")


driver = webdriver.Chrome(options=chrome_options)
driver.get('https://fastdl.app/en')
driver.implicitly_wait(10) 

input_element=driver.find_element(By.CSS_SELECTOR,'input')
driver.implicitly_wait(10)
input_element.click()

driver.implicitly_wait(10) 
input_element.send_keys(video_url)
driver.implicitly_wait(10) 
button = driver.find_element(By.CSS_SELECTOR, 'button.search-form__button[type="submit"]')
button.click()
driver.implicitly_wait(10)
time.sleep(5)

body = driver.find_element("tag name", "body")
print(driver.execute_script("return document.body.scrollHeight"))

# Function to scroll down slowly
def smooth_scroll_down(driver, scroll_pause_time=0.2, pixels_per_scroll=100):
  current_scroll = 0  # Keep track of the current scroll position
  while current_scroll < 10000:
    driver.execute_script(f"window.scrollBy(0, {pixels_per_scroll});")
    current_scroll += pixels_per_scroll
    sleep(scroll_pause_time)

# Call the smooth scrolling function
smooth_scroll_down(driver)

print(driver.execute_script("return document.body.scrollHeight"))

time.sleep(5)
buttons = driver.find_elements(By.CSS_SELECTOR, "a.button.button--filled.button__download")
len(buttons)

In [ ]:

driver.quit()


In [ ]:
buttons[0].get_attribute('href')

In [ ]:
#import img in base 64
import base64
image_path = 'results/frames_DC7PCgOoF2t_2/DC7PCgOoF2t_0.jpg'
with open(image_path, "rb") as image_file:
    base64_string = base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
base64_string

In [ ]:

# Get list of all .jpg files in the directory
import os
directory = 'results/frames_DC7PCgOoF2t_2'
jpg_files = [file for file in os.listdir(directory) if file.endswith(".jpg")]

print("JPG files:", jpg_files)

In [ ]:
from utils.download_content import download_content
from utils.coach_ig_info import coach_ig_info
from utils.mp4_to_jpg import mp4_to_jpg
from utils.img_interpreter import img_interpreter
import os, shutil,base64


#specify the video url
coach_url = 'https://www.instagram.com/emmafituk_/?hl=en'
posts_info = coach_ig_info(coach_url.split('/')[-2])

In [ ]:
posts_info._date

In [ ]:
import subprocess, os, shutil
from moviepy.editor import VideoFileClip
#compute video duration
video_name='DC4_BEyRwbA_0.mp4'
video_filename = 'results/'+video_name
video_filename

In [ ]:
def mp4_to_jpg(video_name = 'DDDEwC4pVWm_0.mp4'):
  from moviepy.editor import VideoFileClip
  import subprocess, os, shutil

  #clean old results
  directory="results/frames_"+video_name[:-6]
  if os.path.exists(directory): shutil.rmtree(directory)
  os.mkdir(directory)

  video_filename = 'results/'+video_name

  #compute video duration
  video = VideoFileClip(video_filename)
  duration = video.duration
  #compute the frames seconds based on the video length
  seconds = [f"00:00:{x:02}" for x in [round(1 + i * (duration - 1) / 5) for i in range(6)]]

  for x,second in enumerate(seconds):
      command = 'ffmpeg -y -ss '+str(second)+' -i  '+video_filename +' -frames:v 1 '+directory+'/frame_'+str(x)+'.jpg'
      subprocess.run(command, shell=True, executable="/bin/bash",stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
      
  return directory

In [ ]:
mp4_to_jpg(video_name='DC4_BEyRwbA_0.mp4')

In [ ]:
os.remove('results/DC4_BEyRwbA_0.mp4')

In [ ]:
directory = 'results/frames_DC4_BEyRwbA'
images=[file for file in os.listdir(directory) if file.endswith(".jpg")]
images

In [ ]:
images_description = 'Summaries of the descriptions of the images in DC4_BEyRwbA content:\n\n'
for i,ip in enumerate(images):
    image_path=directory+'/'+ip
    with open(image_path, "rb") as image_file: base64_string = base64.b64encode(image_file.read()).decode('utf-8')
    img_description = img_interpreter(base64_string)
    images_description = images_description + 'Image_'+str(i)+ ':' +img_description+'\n\n'
    #delete the image
    os.remove(image_path)

In [ ]:
def download_content(video_url,coach_handel):
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.by import By
    import time, os,requests
    from bs4 import BeautifulSoup


    #create results folder if does not exists
    content_name = video_url.split('/')[-2]
    res_folder=os.getcwd()+'/results/'+str(coach_handel)+'/'+str(content_name)
    if os.path.exists(res_folder) and os.path.isdir(res_folder): shutil.rmtree(res_folder)
    os.makedirs(res_folder, exist_ok=True)
    #initiate webdriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Optional: run in headless mode
    prefs = {"download.default_directory" : res_folder}
    chrome_options.add_experimental_option("prefs",prefs)

    driver = webdriver.Chrome(options=chrome_options)
    driver.get('https://fastdl.app/en')
    driver.implicitly_wait(2) 
    
    while True:
        try:
            input_element=driver.find_element(By.CSS_SELECTOR,'input')
            driver.implicitly_wait(3)
            input_element.click()

            driver.implicitly_wait(3) 
            input_element.send_keys(video_url)
            driver.implicitly_wait(3) 
            button = driver.find_element(By.CSS_SELECTOR, 'button.search-form__button[type="submit"]')
            button.click()
            driver.implicitly_wait(3)
            time.sleep(2)
            break
        except:
            time.sleep(0.5)
            pass

    while True:
        try:
            buttons = driver.find_elements(By.CSS_SELECTOR, "a.button.button--filled.button__download")
            button_list=[]
            for button in buttons:
                driver.implicitly_wait(5) 
                button_list.append(button.get_attribute('href'))
            for i,url in enumerate(button_list):
                if '.mp4' in url: extension='.mp4'
                else: extension='.jpg'
                r = requests.get(url)
                results_directory=res_folder+'/'+str(content_name)+'_'+str(i)+extension
                with open(results_directory, 'wb') as f:
                    f.write(r.content)
            Last_created_file=str(content_name)+'_'+str(i)+extension
            if Last_created_file in [f for f in os.listdir(res_folder) if os.path.isfile(os.path.join(res_folder, f))]:
                break
        except:
            time.sleep(0.5)
            pass


    driver.quit()
    files = [res_folder+'/'+f for f in os.listdir(res_folder) if os.path.isfile(os.path.join(res_folder, f))]
    return files

file_names=download_content(video_url='https://www.instagram.com/p/DDAVJ0DoyVc/?hl=en',coach_handel='emmafituk_')

In [ ]:
if '.mp4' in file_names[0]:
    print('ok!')

In [ ]:
file_names[0]


In [ ]:
directory=''
for i,f in enumerate(file_names[0].split('/')):
    if i != len(file_names[0].split('/'))-1:
        directory=directory+f+'/'
directory[:-1]

In [ ]:
from utils.coach_ig_info import coach_handles_all

coach_handles_all_l=coach_handles_all()
coach_urls = ['https://www.instagram.com/'+str(h)+'/?hl=en' for h in coach_handles_all_l.handle]

In [12]:
def assign_category(frames_description):
    import openai
    from openai import OpenAI
    from dotenv import load_dotenv
    import os
    load_dotenv('.env')
    #post categories
    type_description = ''' 
    1. Entertainment: fits if the post seem to be mostly for entertainment purpose and is not directly selling anything nor giving useful tips or the like.
    2. About online coaching: fits if it describes what online coaching is, but is not outright trying to sell it.
    3. Promotion: is for a promotional content where the goal clearly is to try to sell or advertise a product (most likely online coaching or another type of programme).
    4. Educational/Tips: is for content that strives to teach about something be it nutrition, workouts, giving a recipe or other things, without focusing on selling something at the same time.
    5. Testimonials/Social Proof: is for a post that strives to show the effectiveness of something by showing or talking about an example of a person doing that thing - eg. transformation pictures, before and after weight or measurements or quotes from others about how well the thing worked for them.
    6. Mindset/Inspiration: is if the post focuses on one\’s mindset or strives to inspire.
    7. Brand/Personal is for content that is more specific to the sender whether it’s more personal content (eg. about the person writing the post, their friends or family, big events in their life) or very brand specific (eg. presenting an image for the brand, talking about their brand specifically).
    8. Other: is for a post that matches none of the categories listed.
    '''
    #Step5: analyze the video and assign a category
    prompt_video = "Given this definition of content types: " + type_description + '''Please evaluate this Social media content summary and fit it in only one type. 
    it is important that you only answer in the following dictionary format. Do not add anything to your answer: 
    {
    "Date of evaluation": "[Additional info date]",
    "Post date": "[Content Posted date]",
    "Coach name": "[name of the coach]",
    "Content id": "[content identifier]",
    "Post Type": "[Post Type name]",
    "Accuracy Post Type": "[percentage of accuracy]",
    "Comments count": "[number of comments]",
    "Likes": "[number of Likes]",
    "Views": "[number of views]",
    "Reactions": "[number of reactions]",
    "Shares": "[number of shares]",
    "Content short description": "[short summary of the content and explanation of your decision]"
    }
    '''

    api_key = os.getenv('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key)
    message='Content description: '+frames_description
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_video},  # Keep the text prompt as it is
                    {"type": "text", "text": message}  # Include any additional text instead of an image
                ]
            }
        ],
        temperature=0,
        max_tokens=200
    )
    input_tokens = response.usage.prompt_tokens  # Tokens used for the prompt
    output_tokens = response.usage.completion_tokens  # Tokens used for the completion

    # Pricing details for GPT-4o-mini
    input_cost_per_million = 0.15  # USD per million input tokens
    output_cost_per_million = 0.60  # USD per million output tokens
    # Pricing details for GPT-4-turbo
    input_cost_per_million = 10  # USD per million input tokens
    output_cost_per_million = 30  # USD per million output tokens


    # Calculate the cost for input and output tokens
    input_cost = (input_tokens / 1_000_000) * input_cost_per_million
    output_cost = (output_tokens / 1_000_000) * output_cost_per_million
    total_cost = input_cost + output_cost
    cost = f"Total estimated cost: ${total_cost:.4f}"
    return response.choices[0].message.content, cost

In [13]:
with open('results/Full_info_on_runeraahauge.dk.txt', 'r', encoding='utf-8') as file: content = file.read()
assign_category(content)

('{\n    "Date of evaluation": "2024-12-10",\n    "Post date": "2024-12-09",\n    "Coach name": "runeraahauge.dk",\n    "Content id": "DDXnBPwObJP",\n    "Post Type": "Educational/Tips",\n    "Accuracy Post Type": "95%",\n    "Comments count": "0",\n    "Likes": "35",\n    "Views": "3207",\n    "Reactions": "35",\n    "Shares": "0",\n    "Content short description": "The content describes a specific exercise variation, the Supported Elevated Reverse Lunge, detailing its benefits and encouraging others to try it. This aligns with the Educational/Tips category as it aims to teach about fitness without a promotional focus."\n}',
 'Total estimated cost: $0.0076')

In [46]:
import pandas as pd, os, json
coaches_evaluation = txt_files = [f for f in os.listdir('results') if f.endswith('.txt')]
Coaches_content_type = pd.DataFrame()
for i, ce in enumerate(coaches_evaluation):
    with open('results/'+ce, 'r', encoding='utf-8') as file:
        content = file.read()
    Content_type = json.loads(assign_category(content))
    table_t=pd.DataFrame(Content_type,index=[i])
    Coaches_content_type = pd.concat([Coaches_content_type ,table_t])

Coaches_content_type.to_excel('Coaches_content_type.xlsx',index=False)

,Date of evaluation,Post date,Coach name,Content id,Post Type,Accuracy Post Type,Comments count,Likes,Accuracy,Views,Reactions,Shares,Content short description
0,2024-12-10,2024-11-29,abdelloo,DC8zLFFoYuj,Educational/Tips,95%,0,38,95%,7767,38,0,The content discusses common misconceptions ab...
1,2024-12-10,2024-12-09,sofiaakumasci,DDXIfqxNZch,Promotion,90%,2,54,85%,0,54,<NA>,The post promotes a coaching service by showca...
2,2024-12-10,2024-12-09,runeraahauge.dk,DDXnBPwObJP,Educational/Tips,90%,0,35,90%,3207,35,0,The post describes a specific exercise variati...
3,2024-12-10,2024-12-09,jensthebeast1,DDXKJ47s5q7,Educational/Tips,90%,2,30,85%,4473,30,0,The post compares homemade date snacks to Mara...
4,2024-12-10,2024-12-09,mikkeldove,DDXRLwkoif6,Educational/Tips,95%,0,33,95%,4759,33,1,The post provides a detailed explanation of ho...
5,2024-12-10,2024-12-09,borkfitness,DDXONEGoUXf,Educational/Tips,90%,0,84,85%,8551,84,0,The content features a Q&A session discussing ...
6,2024-12-10,2024-12-09,actionanton,DDXFbZ-sveu,Educational/Tips,90%,2,27,85%,<NA>,27,0,The post compares high-calorie muffins from 7-...
7,2024-12-10,2024-12-09,larsencoaching,DDXGyqPNcLz,Promotion,90%,8,126,95%,21343,126,0,The content features a humorous take on daily ...
8,2024-12-10,2024-12-09,fitbywh,DDXz-o5t1av,Promotion,90%,94,30,85%,1175,30,0,The post promotes a specific cheesecake recipe...
9,2024-12-10,2024-12-09,01mathias,DDWsjwmNg9c,Educational/Tips,95%,0,19,95%,3527,19,0,The post provides a recipe for a protein oatme...


In [48]:
Coaches_content_type.to_excel('Coaches_content_type.xlsx',index=False)

In [52]:
Content_type['Post Type']
Content_type['Accuracy Post Type']

'90%'

NameError: name 'content' is not defined